# Mapping exersize
We are going to plot the the county that are listed in the Churn_OD file
1.   Importing the data 
2.   Adjust the coordinates to have more clean data format
3.   Mapping County on map visualisation 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# installing and importin the mapping library
!pip install folium
import folium

#1.Importing the data


In [ ]:
root_URL = 'https://raw.githubusercontent.com/StevenVuong/WDL_2020/master/data/stage1'

Churn_OD_file = 'Churn_OD.txt'
Churn_OD_URL = root_URL +'/'+ Churn_OD_file

County_Coordinates_Data_URL = 'https://raw.githubusercontent.com/Hardi-Lior/WDL/main/Stage_1/data/County_Coordinates_Data.csv'

In [ ]:
df_Churn_OD = pd.read_csv(Churn_OD_URL,delimiter="|",encoding = 'Latin1')
df_County_Coordinates_Data = pd.read_csv(County_Coordinates_Data_URL)

In [ ]:
df_Churn_OD.head()

,Region_of_Origin,District_of_Origin,County_of_Origin,Region_of_Public_Transportation,District_of_Public_Transportation,County_of_Public_Transportation,Dicofre_ParishCode_of_Public_Transportation,Demand_weight
0,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110608,0.307323
1,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110639,0.069997
2,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110658,0.066059
3,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110654,0.059847
4,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110633,0.052341


In [ ]:
df_County_Coordinates_Data.head()

,address,location,point,latitude,longitude,altitude,County
0,"amadora,lisboa,Portugal","Amadora, Lisboa, Grande Lisboa, Área Metropoli...","(38.7595162, -9.223677009460125, 0.0)",38.759516,-9.223677,0.0,amadora
1,"cascais,lisboa,Portugal","Cascais, Lisboa, Grande Lisboa, Área Metropoli...","(38.72240025, -9.396909171649877, 0.0)",38.722400,-9.396909,0.0,cascais
2,"lisboa,lisboa,Portugal","Lisboa, Grande Lisboa, Área Metropolitana de L...","(38.7077507, -9.1365919, 0.0)",38.707751,-9.136592,0.0,lisboa
3,"loures,lisboa,Portugal","Loures, Lisboa, Grande Lisboa, Área Metropolit...","(38.8578642, -9.175816056948255, 0.0)",38.857864,-9.175816,0.0,loures
4,"mafra,lisboa,Portugal","Mafra, Lisboa, Grande Lisboa, Área Metropolita...","(38.9369782, -9.3282374, 0.0)",38.936978,-9.328237,0.0,mafra


## 2.Adjust the coordinates to have more clean data format



In [ ]:
# formating
df_Churn_OD['District_of_Origin'] = df_Churn_OD['District_of_Origin'].str.lower()
df_Churn_OD['County_of_Origin'] = df_Churn_OD['County_of_Origin'].str.lower()

df_Churn_OD['District_of_Public_Transportation'] = df_Churn_OD['District_of_Public_Transportation'].str.lower()
df_Churn_OD['County_of_Public_Transportation'] = df_Churn_OD['County_of_Public_Transportation'].str.lower()


df_Churn_OD.head()

,Region_of_Origin,District_of_Origin,County_of_Origin,Region_of_Public_Transportation,District_of_Public_Transportation,County_of_Public_Transportation,Dicofre_ParishCode_of_Public_Transportation,Demand_weight
0,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,lisboa,110608,0.307323
1,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,lisboa,110639,0.069997
2,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,lisboa,110658,0.066059
3,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,lisboa,110654,0.059847
4,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,lisboa,110633,0.052341


i am looking for number of bus journeys from County to County

In [ ]:
df_Churn_OD["County_County"] = df_Churn_OD["County_of_Origin"] + ' to ' + df_Churn_OD["County_of_Public_Transportation"]
df_journeys = df_Churn_OD.drop_duplicates(subset=['County_County'])
df_journeys.head()

,Region_of_Origin,District_of_Origin,County_of_Origin,Region_of_Public_Transportation,District_of_Public_Transportation,County_of_Public_Transportation,Dicofre_ParishCode_of_Public_Transportation,Demand_weight,County_County
0,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,lisboa,110608,0.307323,amadora to lisboa
6,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,amadora,111512,0.039779,amadora to amadora
12,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,oeiras,111012,0.022297,amadora to oeiras
28,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,loures,110726,0.001094,amadora to loures
31,R1 - AM Lisboa,lisboa,amadora,R1 - AM Lisboa,lisboa,odivelas,111608,0.000722,amadora to odivelas


In [ ]:
df_journeys = df_journeys.drop(columns=['County_County', 'Dicofre_ParishCode_of_Public_Transportation'])
df_journeys.columns

Index(['Region_of_Origin', 'District_of_Origin', 'County_of_Origin',
       'Region_of_Public_Transportation', 'District_of_Public_Transportation',
       'County_of_Public_Transportation', 'Demand_weight'],
      dtype='object')

In [ ]:
## creating dict of df_County_Coordinates_Data with county name and coords
County_lat_lon_df = pd.DataFrame()
County_lat_lon_df['County'] = df_County_Coordinates_Data['County']
County_lat_lon_df['latitude'] = df_County_Coordinates_Data['latitude']
County_lat_lon_df['longitude'] = df_County_Coordinates_Data['longitude']
# County_lat_lon_df.head()
## convert to dict of counties and their coresponding coordinates
County_lat_lon_dict = County_lat_lon_df.set_index('County').T.to_dict('list')

In [ ]:
# we want to plot the df_Churn_OD_slim
df_Churn_OD_slim = df_Churn_OD.loc[:, df_Churn_OD.columns.intersection(['County_of_Origin', 'County_of_Public_Transportation','Demand_weight'])]
df_Churn_OD_slim.head()

,County_of_Origin,County_of_Public_Transportation,Demand_weight
0,amadora,lisboa,0.307323
1,amadora,lisboa,0.069997
2,amadora,lisboa,0.066059
3,amadora,lisboa,0.059847
4,amadora,lisboa,0.052341


In [ ]:
# getting that depature county coordinates and arrivel county coordinates
for idx, val in df_Churn_OD_slim['County_of_Origin'].iteritems():
    df_Churn_OD_slim.at[idx, 'dep_lat'], df_Churn_OD_slim.at[idx, 'dep_lon'] = County_lat_lon_dict[val]
for idx,val in df_Churn_OD_slim['County_of_Public_Transportation'].iteritems():
    df_Churn_OD_slim.at[idx, 'arr_lat'], df_Churn_OD_slim.at[idx, 'arr_lon'] = County_lat_lon_dict[val]
df_Churn_OD_slim.head()

,County_of_Origin,County_of_Public_Transportation,Demand_weight,dep_lat,dep_lon,arr_lat,arr_lon
0,amadora,lisboa,0.307323,38.759516,-9.223677,38.707751,-9.136592
1,amadora,lisboa,0.069997,38.759516,-9.223677,38.707751,-9.136592
2,amadora,lisboa,0.066059,38.759516,-9.223677,38.707751,-9.136592
3,amadora,lisboa,0.059847,38.759516,-9.223677,38.707751,-9.136592
4,amadora,lisboa,0.052341,38.759516,-9.223677,38.707751,-9.136592


In [ ]:
# Sort by ascending county name
df_Churn_OD_slim = df_Churn_OD_slim.sort_values(by=['County_of_Origin'])
df_Churn_OD_slim.head()

,County_of_Origin,County_of_Public_Transportation,Demand_weight,dep_lat,dep_lon,arr_lat,arr_lon
732,alcochete,lisboa,0.592367,38.75605,-8.960844,38.707751,-9.136592
756,alcochete,lisboa,0.000076,38.75605,-8.960844,38.707751,-9.136592
755,alcochete,porto,0.000077,38.75605,-8.960844,41.149451,-8.610788
754,alcochete,porto,0.000077,38.75605,-8.960844,41.149451,-8.610788
753,alcochete,lisboa,0.000085,38.75605,-8.960844,38.707751,-9.136592


#3.   Mapping County on map visualisation 

In [ ]:
map1 = folium.Map(
    location=[38.7223,-9.1393],
    tiles='cartodbpositron',
    zoom_start=11,
    min_zoom=4,
    max_zoom=14
)
df_County_Coordinates_Data.apply(lambda row:folium.Marker(location=[row["latitude"], row["longitude"]],popup=row["address"]).add_to(map1), axis=1)


0     <folium.map.Marker object at 0x7feeb02d5310>
1     <folium.map.Marker object at 0x7feeb027d590>
2     <folium.map.Marker object at 0x7feeb027d0d0>
3     <folium.map.Marker object at 0x7feeb065b2d0>
4     <folium.map.Marker object at 0x7feeb065b590>
5     <folium.map.Marker object at 0x7feeb0666510>
6     <folium.map.Marker object at 0x7feeb02d1650>
7     <folium.map.Marker object at 0x7feeb02d1890>
8     <folium.map.Marker object at 0x7feeb02d19d0>
9     <folium.map.Marker object at 0x7feeb02d1a90>
10    <folium.map.Marker object at 0x7feeb02df2d0>
11    <folium.map.Marker object at 0x7feeb02dfb10>
12    <folium.map.Marker object at 0x7feeb02dfd50>
13    <folium.map.Marker object at 0x7feeb02dfb90>
14    <folium.map.Marker object at 0x7feeb0551150>
15    <folium.map.Marker object at 0x7feeb0566cd0>
16    <folium.map.Marker object at 0x7feeb0239c10>
17    <folium.map.Marker object at 0x7feeb0282f50>
18    <folium.map.Marker object at 0x7feeb56ca2d0>
19    <folium.map.Marker object

In [ ]:
path_coords = [[1,2],[3,4]]
for val in range(len(df_Churn_OD_slim)):
  Feature_Group_Name = 'journy_from_' + str(df_Churn_OD_slim.County_of_Origin[val])
  Name = df_Churn_OD_slim.County_of_Origin[val]
  line = 'line_' + str(count)
  path_coords[0][0] = df_Churn_OD_slim.dep_lat[val]
  path_coords[0][1] = df_Churn_OD_slim.dep_lon[val]
  path_coords[1][0] = df_Churn_OD_slim.arr_lat[val]
  path_coords[1][1] = df_Churn_OD_slim.arr_lon[val]
  # Creating feature groups
  Name = folium.FeatureGroup(Feature_Group_Name)
  # Adding lines to the different feature groups
  line_1 =folium.vector_layers.PolyLine(path_coords,popup=Feature_Group_Name,tooltip=Feature_Group_Name,weight=5).add_to(Name)
  Name.add_to(map1)



In [ ]:
folium.LayerControl().add_to(map1)

In [ ]:
map1